# Dashboard

In this notebook, we'll develop a dashboard for explanations. We'll use
the explanations that were output by the Amazon SageMaker Batch Transform
Job performed in the last stage. Our dashboard is created using a Python
library called [Streamlit](https://www.streamlit.io/).

<p align="center">
  <img src="https://github.com/awslabs/sagemaker-explaining-credit-decisions/raw/master/docs/architecture_diagrams/stage_5.png" width="1000px">
</p>

We start by importing a variety of packages that will be used throughout
the notebook.

In [ ]:
import json
from pathlib import Path
import sagemaker

Up next, we define a couple of functions to construct the correct URL for
dashboard that will be running locally on the Amazon SageMaker Notebook
Instance. When running a server on a local port, it must be accessed via
the [Jupyter Server
Proxy](https://github.com/jupyterhub/jupyter-server-proxy) at
`https://{notebook-url}/proxy/{port}`.

In [ ]:
def get_notebook_name():
    with open('/opt/ml/metadata/resource-metadata.json') as openfile:
        data = json.load(openfile)
    notebook_name = data['SolutionPrefix']
    return notebook_name


def get_dashboard_url(port):
    notebook_name = get_notebook_name()
    region_name = sagemaker.Session().boto_region_name
    return f"https://{notebook_name}.notebook.{region_name}.sagemaker.aws/proxy/{port}/"

We define the port on which to run the server, obtain the dashboard URL
(for that port) and start the Streamlit server. Code for the Streamlit
application can be found (and edited) in `../containers/dashboard/src`.
We're using the file called `app.py` as our dashboard entry point. After
running the cell below, click on the `Dashboard URL` link that appears,
and the running dashboard should open in a new browser tab.

In [ ]:
port = 8501
url = get_dashboard_url(port)
!echo Dashboard URL: {url}
!(cd ../containers/dashboard/src && streamlit run app.py --server.port {port})

When you're finished exploring the dashboard, you can interupt the kernel
in this notebook to stop the Streamlit server that is running in the cell
above.

## Deployment

Although dashboard access could in theory be managed through IAM access control to this Amazon SageMaker Notebook Instance, it's much more robust to deploy the dashboard elsewhere using services such as Amazon Elastic Container Service (ECS). As a first stepping stone to dashboard deployment, we define and build a Docker image for the dashboard. We copy the `package` and `.env` file so that can be used inside the container (see [Dockerfile](/edit/sagemaker-explaining-credit-decisions/sagemaker/containers/dashboard/Dockerfile#)).

In [ ]:
image_name = "dashboard"
!rm -rf ../containers/dashboard/package
!cp -r ../package ../containers/dashboard/
!cp -r ../.env ../containers/dashboard/
!(cd ../containers/dashboard && docker build -t {image_name} .)

After the build has completed successfully, you can test the container directly on this Amazon SageMaker Notebook Instance. We define a function to construct the correct `docker run` command to forward ports, mount local directories (optional) and pass through IAM credentials.

In [ ]:
def get_docker_run_command(port, image, local_dir_mount=False):
    session = sagemaker.Session()
    region_name = session.boto_region_name
    credentials = session.boto_session.get_credentials()
    command = [f"docker run -p {port}:80"]
    if local_dir_mount:
        local_dir_mount = Path(local_dir_mount).resolve()
        command += [f"-v {local_dir_mount}:/usr/src/app/src"]
    command += [
        f"--env AWS_DEFAULT_REGION={region_name}",
        f"--env AWS_ACCESS_KEY_ID={credentials.access_key}",
        f"--env AWS_SECRET_ACCESS_KEY={credentials.secret_key}",
        f"--env AWS_SESSION_TOKEN={credentials.token}",
        f"{image}"
    ]
    return " \\\n".join(command)

You can then use the function to start the dashboard container. Uncomment `local_dir_mount` to mount the local `src` directory if you wish to develop dashboard code and test without having to rebuild the container each time you make a change. After
running the cell below, click on the `Dashboard URL` link that appears,
and the running dashboard should open in a new browser tab.

In [ ]:
url = get_dashboard_url(port)
command = get_docker_run_command(port, image_name) # local_dir_mount='../containers/dashboard/src')
!echo Dashboard URL: {url}
!{command}

We don't cover the complete dashboard deployment in this solution, but we have a
[dedicated
solution](https://github.com/awslabs/sagemaker-dashboards-for-ml) that
showcases how to deploy a dashboard on Amazon Elastic
Container Service (ECS). Additional features such as Amazon Cognito
authentication are included too for username/password protected
dashboards.

## Customization

We have provided an example dashboard, but our solution is customizable
if you have your own dashboard requirements. You should edit the contents
of the `../containers/dashboard` directory for this. Additional
dependencies can be added to the `requirements.txt` file or the
`Dockerfile`, and the dashboard source code can be modified in the `src`
folder. Of particular note is the `app.py` file, since it acts as the
entry point to the dashboard. Check out the [Streamlit
Docs](https://docs.streamlit.io/en/latest/) for more details on how to
customize this dashboard. 

## Next Stage

Up next we'll wrap things up and discuss how to clean up the solution.

[Click here to continue.](./6_conclusion.ipynb)